Purpose of this notebook is to take a csv file from the allen institute (already subsetted to include just results we want for a primary injection site), subset further as needed, and run some grouped analyses. This may be best to do not-in-python long-term but we're starting here.

written by Emily Jane Dennis September 2020, during covid pandemic!

inputs 
    - requires a csv file with Allen data **TODO** add link
    - and a series of extracted folders from Allen (links in spreadsheet)
    - starting with grid ones, tbd
   

In [1]:
import os, sys, time
import tifffile as tif
from scipy.ndimage.interpolation import zoom
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [2]:
# location of your repo clone
sys.path.append("/home/emilyjanedennis/Desktop/GitHub/rat_BrainPipe")

In [3]:
# base path
src = "/home/emilyjanedennis/Desktop/allen_for_PPC/"

In [ ]:
# if you need to generate a subset from the Allen csv
fulldatasets = pd.read_csv(os.path.join(src,"mouse_projection_data_sets.csv"))
ppc = fulldatasets[fulldatasets.primary_injection_structure_acronym=="PTLp"]
ppc.to_csv(os.path.join(src,"ppc.csv"))

# else uncomment the following line if you've already run this step
# ppc = pd.read_csv(os.path.join(src,"ppc.csv"))

In [10]:
listsrc = os.listdir(os.path.join(src,"grids/"))

In [16]:
ppconly = ppc[ppc.secondary_injection_acronym=="{PTLp}"]

In [23]:
filenames=[]

for i in ppconly.index:
    filenames.append(ppconly.grid_file_url[i])

In [51]:
ppconlyfilenames=[]
for file in filenames:
    ppconlyfilenames.append(os.path.join(src,"{test}_grid".format(test=file[-18:-9])))

In [52]:
ppc1=tif.imread(ppconlyfilenames[0])

['/home/emilyjanedennis/Desktop/allen_for_PPC/112229103_grid',
 '/home/emilyjanedennis/Desktop/allen_for_PPC/157769139_grid',
 '/home/emilyjanedennis/Desktop/allen_for_PPC/168165712_grid',
 '/home/emilyjanedennis/Desktop/allen_for_PPC/287225041_grid',
 '/home/emilyjanedennis/Desktop/allen_for_PPC/287601808_grid',
 '/home/emilyjanedennis/Desktop/allen_for_PPC/292792724_grid',
 '/home/emilyjanedennis/Desktop/allen_for_PPC/293821389_grid',
 '/home/emilyjanedennis/Desktop/allen_for_PPC/294533406_grid']